In [53]:
import os
from glob import glob

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
plt.rcParams['figure.figsize'] = (16, 9)
sns.set(style="whitegrid")

llah = glob('data/llah_raw_samples/*.txt')

In [55]:
def calaculate_fixation(fixations):
    df = pd.DataFrame(columns=['x', 'y', 'dur'])
    for i, fix in enumerate(fixations):
        x = np.median(fix[:, 0]).astype(int)
        y = np.median(fix[:, 1]).astype(int)
        dur = len(fix) * 24
        df.loc[i] = [x, y, dur]
    return df

        
def fix_dispersion(gaze_points=None, time_threshold=4, dist_threshold=50):
    fixations = []
    ix = 0
    fix = []
    for i, item in gaze_points.iterrows():
        if i != 0:
            a = np.array([df.loc[ix, 'x'], df.loc[ix, 'y']])
            if len(fix) == 0:
                fix.append(a.tolist())
            b = np.array([item.x, item.y])
            dist = np.linalg.norm(a-b)
            if dist < dist_threshold:
                fix.append(b.tolist())
            else:
                if i - ix >= time_threshold:
                    fixations.append(np.array(fix))
                ix = i
                fix = []
    return calaculate_fixation(fixations)
        

In [56]:
for i, path in enumerate(llah):
    df = pd.read_csv(llah[0], sep='\t', names=['ts', 'x', 'y', 'fix', 'x_fix', 'y_fix', 'fix_counter', 'nearest_word', 'doc' ], header=0)
    df['t'] = df.index
    df = df[['t', 'x', 'y']]
    con = str(i%2+1)
    sub = str(i//2+1)
    if len(sub) == 1:
        sub = '0' + sub
    df.to_csv(f'data/raw_gaze/G'+ sub + '-' + con  + '.csv', sep=',', index=None)
    fix = fix_dispersion(gaze_points=df)
    fix.to_csv(f'data/fixations/F'+ sub + '-' + con  + '.csv', sep=',', index=None)
